# Seq2Seq Builder


In this notebook we will see how to build a Seq2Seq model, to construct the source code of the seq2seq model, and also drop anything useless.

For that, we will use the following imports:
- from seq2seq.data import SeqDataset
- from seq2seq.embeddings import OneHotEmbedding
- from seq2seq.parser import parser

In [35]:
from src.seq2seq.dataset import SeqDataset 

In [1]:
from src.seq2seq import train
from src.seq2seq.model import ModelConfig
from types import SimpleNamespace

# Mantener tus argumentos como los tenías
args = SimpleNamespace(
    device='cpu',
    batch_size=4,
    max_length=512,
    num_workers=2,
    quiet=False,
    command='train',
    train_file='data/ArchiveII.csv',
    valid_file=None,
    out_path='./build_test/build_AE_seq2seq.csv',
    max_epochs=1,
    no_cache=False
)

# Crear el diccionario de configuración usando ModelConfig como base
config = vars(ModelConfig(
    device=args.device,
    verbose=not args.quiet
))

# Agregar los parámetros adicionales que no son parte de ModelConfig
config.update({
    "batch_size": args.batch_size,
    "max_len": args.max_length,
    "cache_path": None if args.no_cache else "cache/",
    "max_epochs": args.max_epochs
})

# Llamar a train con los argumentos
train(
    train_file=args.train_file,
    config=config,
    out_path=args.out_path,
    valid_file=args.valid_file,
    nworkers=args.num_workers
)

Working on ./build_test/build_AE_seq2seq.csv


ValueError: Output path ./build_test/build_AE_seq2seq.csv already exists

In [62]:
from torch.utils.data import DataLoader
from src.seq2seq.dataset import SeqDataset, pad_batch
    
train_loader = DataLoader(
    SeqDataset(args.train_file, training=True, **config),
    batch_size=args.batch_size, 
    shuffle=True,
    num_workers=args.num_workers,
    collate_fn=pad_batch
)

In [66]:
len(train_loader)

966